In [1]:
import intrepid as ip
import intrepid.utils
import intrepid.plots
import imp
import pprint

f, filename, description = imp.find_module('traffic_light')
traffic_light = imp.load_module('TrafficLight', f, filename, description)

ctx = ip.mk_ctx()

myTrafficLight = traffic_light.SimulinkCircuit(ctx, 'MyTrafficLight')
myTrafficLight.mk_circuit()

lightOut = myTrafficLight.outputs['traffic_light/out']
targetGreen = ip.mk_eq(ctx, lightOut, myTrafficLight.nets['traffic_light/Green'])
targetYellow = ip.mk_eq(ctx, lightOut, myTrafficLight.nets['traffic_light/Yellow'])
targetRed = ip.mk_eq(ctx, lightOut, myTrafficLight.nets['traffic_light/Red'])
targetOff = ip.mk_eq(ctx, lightOut, myTrafficLight.nets['traffic_light/Off'])

bmc = ip.mk_engine_bmc(ctx)
ip.bmc_add_target(ctx, bmc, targetGreen)
ip.bmc_add_target(ctx, bmc, targetYellow)
ip.bmc_add_target(ctx, bmc, targetRed)
ip.bmc_add_target(ctx, bmc, targetOff)

pprint.pprint(myTrafficLight.outputs)

for depth in range(4):
    reachedTargets = ip.utils.bmc_reach_at_depth(ctx, bmc, depth)
    if len(reachedTargets) > 0:
        print 'Targets reached at depth', depth, ':', len(reachedTargets)
        cex = ip.bmc_get_counterexample(ctx, bmc, reachedTargets[0])
        cexDict = ip.utils.counterexample_get_as_dictionary(ctx, cex, myTrafficLight.inputs, myTrafficLight.outputs)
        # pprint.pprint(cexDict, indent=1)
        ip.plots.plot_counterexample_dictionary(cexDict)
    else:
        print 'No target reacheable at depth:', depth

ip.del_ctx(ctx)

OrderedDict([('traffic_light/out', 66)])
Targets reached at depth 0 : 1
x: [0, 1]
y: [0, 0]
x: [0, 1]
y: [0, 0]
x: [0, 1]
y: [0, 0]
x: [0, 1]
y: [0, 0]
x: [0, 1]
y: ['#x00', '#x00']


TypeError: cannot concatenate 'str' and 'int' objects